In [1]:
import pandas as pd
import numpy as np

In [2]:
# Importando os dados
df_vendas = pd.read_csv('../data/raw/sales.csv', index_col=0)
df_estoque_lvl = pd.read_csv('../data/raw/sensor_stock_levels.csv', index_col=0)
df_temperatura = pd.read_csv('../data/raw/sensor_storage_temperature.csv', index_col=0)

### Tratando o timestamp
Conforme visto na documentação, as tabelas podem ser unidas usando o timestamp, mas essa variável é medida de forma diferente nas tabelas, dificultando assim a mesclagem. Para resolver isso irei alterar o timestamp apenas para data e hora sem minutos, pois como definido no problema, os sensores realizarão medidas hora a hora.

Como a transformação funcionará:

06/05/2020 19:45 -> 06/05/2020 19:00
09/04/2019 16:10 -> 09/04/2019 16:00
19/10/2021 09:30 -> 19/10/2021 09:00

In [3]:
# Alterando a coluna timestamp
df_vendas['timestamp'] = pd.to_datetime(df_vendas.timestamp.str.slice(0, 16))
df_estoque_lvl['timestamp'] = pd.to_datetime(df_estoque_lvl.timestamp.str.slice(0, 16))
df_temperatura['timestamp'] = pd.to_datetime(df_temperatura.timestamp.str.slice(0, 16))


### Unindo as tabelas

In [4]:
# Unindo as tabelas e verificando o shape
df_vendas.merge(df_estoque_lvl, on = ['product_id', 'timestamp']).shape

(92, 11)

In [5]:
# Calculando a perda de dados
1 - df_vendas.merge(df_estoque_lvl, on = ['product_id', 'timestamp']).shape[0]/df_vendas.shape[0]

0.9882488184953379

Como a data e hora das vendas são diferentes das datas e hora das medições dos sensores, a união realizada pelo método **merge** está causando uma perda de cerca de 98% dos dados de vendas, o que é bastante.

Para solucionar esse problema, iremos unir os datasets manualmente utilizando o id e data, onde um produto será unido ao percentual de estoque anterior ao timestamp da venda.

In [6]:
# Criando lista vazia
lista_prcnt_estoque = []

# Criando um loop para iterar sob cada indice
for num in range(0, df_vendas.shape[0]):

    # Tentando unir os dados usando o ID e o Timestamp
    df_filtrado = df_estoque_lvl.query(f"timestamp == '{str(df_vendas.iloc[num, 1])}' and product_id == '{df_vendas.iloc[num, 2]}'")

    # Definindo um condicional para caso a união anterior não seja realizada
    if df_filtrado.shape[0] == 0:

       # Buscando os timestamp referentes ao ID do produto na tabela de estoque
        timestamp_estoque = df_estoque_lvl.query(f"product_id == '{df_vendas.iloc[num, 2]}'").timestamp.to_list()

       # Buscando os timestamp referentes ao ID do produto na tabela de vendas e adicionando a uma lista
        timestamp_venda = df_vendas.iloc[num, 1]
        timestamp_estoque.append(timestamp_venda)

        # Ordenando as variáveis
        timestamp_ordenado = sorted(timestamp_estoque)

       # Obtendo a data anterior a da venda e salvando em uma variável
        index_timestamp = timestamp_ordenado.index(timestamp_venda) - 1
        timestamp_novo = timestamp_ordenado[index_timestamp]

       # Buscando a pctg de estoque usando o ID e o novo Timestamp
        df_filtrado = df_estoque_lvl.query(f"timestamp == '{timestamp_novo}' and product_id == '{df_vendas.iloc[num, 2]}'")
        lvl_estoque = df_filtrado.estimated_stock_pct.values[0]

        # Adicionando a uma lista
        lista_prcnt_estoque.append(lvl_estoque)

    # Juntando as tabelas normalmente caso o condicional não seja atendido
    else:
        lvl_estoque = df_filtrado.estimated_stock_pct.values[0]
        lista_prcnt_estoque.append(lvl_estoque)

In [7]:
# Adicionando os valores ao dataset de vendas
df_vendas['prcnt_stock'] = lista_prcnt_estoque

In [8]:
# Checando a próxima tabela
df_temperatura.sort_values('timestamp')

,id,timestamp,temperature
14130,d451bd29-d3d2-42a1-b228-150b5ba4d664,2022-03-01 09:00:00,-1.93
15982,148a729c-0926-4f8b-9bda-f214d0f67b8d,2022-03-01 09:00:00,1.00
5593,151f56ba-b488-4297-b503-f5b50fee2be7,2022-03-01 09:00:00,-2.10
6061,d050c5f2-0ffc-49ee-b8a0-beef559f5b29,2022-03-01 09:00:00,1.85
7495,fb02fefe-da3a-4e1d-98be-f6523bc6fac6,2022-03-01 09:00:00,-2.45
...,...,...,...
14453,4a978e83-4b25-4648-85c5-f354e015fe4d,2022-03-07 19:59:00,-2.91
7902,36a6cdb6-b344-4bfb-ab21-4f33ffe064e0,2022-03-07 19:59:00,0.71
18379,d502a503-33ec-49c8-97e7-792aa6ce3421,2022-03-07 19:59:00,-2.98
12147,33313da5-9999-4aef-a595-63ad14fe9468,2022-03-07 19:59:00,0.55


Aqui a nossa abordagem será alterada, pois ao contrário da tabela de estoque, onde tínhamos valores únicos por timestamp devido a diferenciação proporcionada ID, não possuímos a mesma diferenciação aqui, onde cada timestamp possui vários valores diferentes de temperatura.

Sendo assim, irei agrupar o timestamp e calcular algumas estatísticas para cada horário, como média, desvio padrão, mediana, variância, etc. É necessário valores únicos para uma união sem perda de valor entre as tabelas.

In [9]:
# Computando estatísticas e resetando o index
df_temperatura = df_temperatura.groupby('timestamp').temperature.agg(['std', 'var', 'sem', 'mean', 'median', 'min', 'max'])
df_temperatura = df_temperatura.reset_index()

In [13]:
# Unindo as tabelas
df_final = df_vendas.merge(df_temperatura, on = 'timestamp')